In [1]:
source = "CombineSignals";

In [2]:
using NBInclude
import XGBoost
@nbinclude("Alpha.ipynb");

In [3]:
@nbinclude("../TrainingAlphas/XGBoostFeatures.ipynb");

In [4]:
# used for XGBoostFeatures
function read_params(alpha)
    params = load("../../data/alphas/$alpha/params.jld2")
    if alpha == "UserItemBiases"
        μ = mean(read_recommendee_alpha("UserItemBiases").rating - params["a"])
        push!(params["u"], μ)
    end
    params
end;

In [5]:
function compute_alpha()
    params = read_params(source)
    X = get_augmented_indep("recommendee_inference", params["alphas"], params["β"])
    bst = XGBoost.Booster(model_file = params["bst"])
    boost_preds = XGBoost.predict(bst, X)
    predictions =
        get_indep("recommendee_inference", params["alphas"]) * params["β"] + boost_preds
    @debug "average bst magnitude: $(mean(abs.(boost_preds))), $(sqrt(mean(boost_preds .^ 2)))"

    # predictions
    model(items) = predictions[items]
    write_recommendee_alpha(model)

    # error predictions
    error_bst = XGBoost.Booster(model_file = params["error_bst"])
    error_preds = clamp.(XGBoost.predict(error_bst, hcat(X, boost_preds)), 0, Inf)
    error_model(items) = error_preds[items]
    write_recommendee_alpha(error_model, outdir = "PredictedErrors")
end

compute_alpha (generic function with 1 method)

In [6]:
compute_alpha()

Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.50 μs/it)it)
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.19 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:06 ( 0.68 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (95.15 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:05 ( 0.61 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:06 ( 0.68 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (84.91 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:07 ( 0.78 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (87.28 μs/it)
[ Debug: 20220320 04:16:30 nsfw categories: String7["gray", "white"]
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.21 ms/it)
[ Debug: 20220320 04:16:31 medium categories: String7["movie", "music", "ona", "ova", "special", "tv", "unknown"]
[ Debug: 20220320 04:16:32 source categories: String15["4_koma_manga", "book", "card_game", "d